In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, Lambda, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow.keras.backend as K

In [ ]:
data_dir = '/CAPSTONE-PROJECT/data/aug/'

In [4]:
def load_and_preprocess_data(data_dir):
    X = []
    y = []

    # Baca data dari direktori aug
    for person_dir in os.listdir(data_dir):
        person_path = os.path.join(data_dir, person_dir)
        if os.path.isdir(person_path):
            person_id = int(person_dir.split('_')[-1])
            image_paths = glob(os.path.join(person_path, '*.jpg'))

            for image_path in image_paths:
                image = cv2.imread(image_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                image = cv2.resize(image, (128, 128))
                image = image / 255.0  # Normalisasi ke rentang 0-1
                X.append(image)
                y.append(person_id)

    return np.array(X), np.array(y)


In [5]:
# Baca dan preprocess data
X, y = load_and_preprocess_data(data_dir)
print(f"Jumlah data: {len(X)}")
print(f"Ukuran input: {X.shape[1:]}")

Jumlah data: 200
Ukuran input: (128, 128)


In [6]:
img_width, img_height = 128, 128

In [7]:
# Fungsi untuk memuat gambar dan resize
def load_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Membaca gambar grayscale
    img = cv2.resize(img, (img_width, img_height))  # Resize gambar
    img = img.astype('float32') / 255.0  # Normalisasi
    img = np.expand_dims(img, axis=-1)  # Menambahkan dimensi channel
    return img

'''def load_image(file_path, target_size=(128, 128)):
    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, target_size)
    image = image.astype("float32") / 255.0  # Normalisasi
    return np.expand_dims(image, axis=-1)'''

'def load_image(file_path, target_size=(128, 128)):\n    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)\n    image = cv2.resize(image, target_size)\n    image = image.astype("float32") / 255.0  # Normalisasi\n    return np.expand_dims(image, axis=-1)'

In [8]:
def create_pairs(directory):
    pairs = []
    labels = []
    people = os.listdir(directory)

    for person in people:
        person_path = os.path.join(directory, person)
        images = os.listdir(person_path)
        for i in range(len(images)):
            for j in range(i + 1, len(images)):
                img1 = load_image(os.path.join(person_path, images[i]))
                img2 = load_image(os.path.join(person_path, images[j]))
                pairs.append([img1, img2])
                labels.append(1)  # Pair positif (gambar dari orang yang sama)

        for other_person in [p for p in people if p != person]:
            other_path = os.path.join(directory, other_person)
            other_image = load_image(os.path.join(other_path, os.listdir(other_path)[0]))
            pairs.append([img1, other_image])
            labels.append(0)  # Pair negatif (gambar dari orang berbeda)

    return np.array(pairs), np.array(labels)

In [9]:
def build_siamese_model(input_shape=(128, 128, 1)):
    input = Input(shape=input_shape)
    x = Conv2D(64, (10, 10), activation='relu')(input)
    x = MaxPooling2D()(x)
    x = Conv2D(128, (7, 7), activation='relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(128, (4, 4), activation='relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(256, (4, 4), activation='relu')(x)
    x = Flatten()(x)
    x = Dense(4096, activation='sigmoid')(x)
    model = Model(input, x)
    return model


In [10]:
def build_full_model(input_shape=(128, 128, 1)):
    base_model = build_siamese_model(input_shape)

    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)

    processed_a = base_model(input_a)
    processed_b = base_model(input_b)

    # Lambda layer untuk menghitung jarak L1
    distance = Lambda(lambda embeddings: tf.abs(embeddings[0] - embeddings[1]))([processed_a, processed_b])
    output = Dense(1, activation="sigmoid")(distance)
    model = Model([input_a, input_b], output)
    return model


In [11]:
model = build_full_model()
model.compile(loss="binary_crossentropy", optimizer=Adam(0.0001), metrics=["accuracy"])
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 128, 128, 1)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_2             │ (None, 128, 128, 1)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional (Functional)   │ (None, 4096)           │     68,307,776 │ input_layer_1[0][0],   │
│                           │                        │                │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda (Lambda)           │ (None, 4096)           │              0 │ functional[0][0],      │
│                           │                        │                │ functional[1][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 1)              │          4,097 │ lambda[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 68,311,873 (260.59 MB)

 Trainable params: 68,311,873 (260.59 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Load pasangan data
pairs, labels = create_pairs(data_dir)

# Split data menjadi training dan validation (contoh 80-20 split)
from sklearn.model_selection import train_test_split
(pairs_train, pairs_val, labels_train, labels_val) = train_test_split(pairs, labels, test_size=0.2, random_state=42
                                                                      )

In [15]:
# Callback untuk early stopping jika val_accuracy tidak meningkat
early_stopping = EarlyStopping(
    monitor="val_accuracy",
    patience=3,
    restore_best_weights=True
)

# Callback untuk mengurangi learning rate jika val_loss stagnan
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=2,
    min_lr=1e-6,
    verbose=1
)

In [ ]:
# Training model
history = model.fit(
    [pairs_train[:, 0], pairs_train[:, 1]], labels_train[:],
    validation_data=([pairs_val[:, 0], pairs_val[:, 1]], labels_val[:]),
    batch_size=16,
    epochs=50,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 857s 17s/step - accuracy: 0.9310 - loss: 0.5560 - val_accuracy: 0.9447 - val_loss: 0.4252 - learning_rate: 1.0000e-04
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 892s 18s/step - accuracy: 0.9550 - loss: 0.3948 - val_accuracy: 0.9447 - val_loss: 0.2964 - learning_rate: 1.0000e-04
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 906s 18s/step - accuracy: 0.9511 - loss: 0.3260 - val_accuracy: 0.9447 - val_loss: 0.3100 - learning_rate: 1.0000e-04
Epoch 4/50
46/50 ━━━━━━━━━━━━━━━━━━━━ 1:04 16s/step - accuracy: 0.9514 - loss: 0.3111

In [ ]:
def plot_scores(train):
    accuracy = train.history['accuracy']
    val_accuracy = train.history['val_accuracy']
    epochs = range(len(accuracy))
    plt.plot(epochs, accuracy, 'b', label='Score apprentissage')
    plt.plot(epochs, val_accuracy, 'r', label='Score validation')
    plt.title('Scores')
    plt.legend()
    plt.show()
def plot_loss(train):
    loss = train.history['loss']
    val_loss = train.history['val_loss']
    epochs = range(len(loss))
    plt.plot(epochs, loss, 'b', label='Loss apprentissage')
    plt.plot(epochs, val_loss, 'r', label='Loss validation')
    plt.title('Loss')
    plt.legend()
    plt.show()

plot_scores(history)
plot_loss(history)

In [ ]:
'''# Contrastive loss
def contrastive_loss(y_true, y_pred):
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)'''